In [37]:
import pandas as pd
import numpy as np
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
import jieba

In [38]:
train_dataset = pd.read_csv('sohu.csv')
train_dataset

,label,text
0,pic,高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑...
1,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
2,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
3,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
4,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...
5,pic,10月4日十佳图：李娜小德球迷抢眼 大威显文静责编:马克杰 日期:2013-10-4 小德的...
6,pic,高清：小德速胜高举双臂庆祝 鼓掌向球迷们致意责编:马克杰 日期:2013-10-4 小德鼓掌...
7,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...
8,news,美“宙斯盾”系统成功进行反导试验新华网华盛顿10月4日电(记者林小春)美国国防部4日宣布再次...
9,news,"3中国建筑工俄罗斯身亡【3中国建筑工俄罗斯身亡】俄伊尔库茨克州称,4日,1名中国建筑工人从在..."


In [39]:
train_dataset['label'].value_counts()

news        2989
sports      1200
business    1051
pic          312
yule         185
mil           95
caipiao       45
cul           44
Name: label, dtype: int64

In [40]:
train_dataset = train_dataset[train_dataset['label'] != 'cul']
train_dataset = train_dataset[train_dataset['label'] != 'caipiao']
train_dataset = train_dataset[train_dataset['label'] != 'mil']
train_dataset = train_dataset[train_dataset['label'] != 'yule']
train_dataset = train_dataset[train_dataset['label'] != 'pic']

In [41]:
train_dataset['label']=train_dataset['label'].map({'news': 0, 'sports': 1, 'business' : 2})

In [42]:
for i in train_dataset['text']:
    i = jieba.cut(i, cut_all = True)

In [43]:
train_dataset['text'] = train_dataset['text'].apply(lambda x:' '.join(jieba.cut(x)))

In [44]:
train_dataset

,label,text
1,0,河南 周口 路边 秸秆 燃烧 2 岁 幼儿 被 熏 晕 ( 图 ) 10 月 3...
2,0,消息 称 138 名 中国 劳工 在 菲律宾 被 抓 使馆 正 核实 【 使馆 正 核实...
3,0,越南 重要 军事 领导人 武元甲 大将 去世 享年 102 岁 据 美国 媒体 10 月...
4,0,西沙 搜救 已 发现 6 具 遇难 渔民 遗体 尚有 52 人 失踪 【 已 发现 6 ...
7,1,卡努特 伤愈 将 首发 战辽足 斯塔诺 ： 辽足 不好 对付 高清 : 国安 踩场 卡努...
8,0,美 “ 宙斯 盾 ” 系统 成功 进行 反导 试验 新华网 华盛顿 10 月 4 日电 ( ...
9,0,3 中国 建筑 工 俄罗斯 身亡 【 3 中国 建筑 工 俄罗斯 身亡 】 俄 伊尔库茨克州...
10,0,山西 煤矿 透水 事故 排水 5 万立方米 仍 有 12 人 被困 中新社 汾阳 10 ...
11,1,现场 ： 彭帅 暴力 发球 遭 无视 主裁 秀 中文 “ 谢谢 ” 北京 时间 9 月 ...
12,0,扬州 警方 ： 一名 游客 在 扬州 博物馆 自主 坠楼 身亡 【 一 游客 在 扬州 坠亡...


In [45]:
MAX_SEQUENCE_LENGTH = 100 # 每条新闻最大长度
EMBEDDING_DIM = 200 # 词向量空间维度
VALIDATION_SPLIT = 0.16 # 验证集比例
TEST_SPLIT = 0.2 # 测试集比例

In [46]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_dataset['text'])
sequences = tokenizer.texts_to_sequences(train_dataset['text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(train_dataset['label']))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 102585 unique tokens.
Shape of data tensor: (5240, 100)
Shape of label tensor: (5240, 3)


In [47]:
p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]
print('train docs: '+str(len(x_train)))
print('val docs: '+str(len(x_val)))
print('test docs: '+str(len(x_test)))

train docs: 3353
val docs: 839
test docs: 1048


In [48]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
batch_size = 128
epochs = 3
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val,y_val))

Train on 3353 samples, validate on 839 samples
Epoch 1/3
3353/3353 [==============================] - 12s 3ms/step - loss: 0.1772 - acc: 0.9355 - val_loss: 0.4076 - val_acc: 0.8371
Epoch 2/3
3353/3353 [==============================] - 12s 3ms/step - loss: 0.0896 - acc: 0.9697 - val_loss: 0.3501 - val_acc: 0.8613
Epoch 3/3
3353/3353 [==============================] - 12s 3ms/step - loss: 0.0632 - acc: 0.9782 - val_loss: 0.3449 - val_acc: 0.8673


In [52]:
accuracy = model.evaluate(x_test, y_test, batch_size = 50)
print("test accuracy:{}".format(accuracy[1]))

1048/1048 [==============================] - 0s 399us/step
test accuracy:0.870228995348661
